In [ ]:
%load_ext watermark
%watermark -a 'Christian Schuhegger' -u -d -v -p numpy,xarray,scipy,pandas,sklearn,matplotlib,seaborn,qgrid,rpy2,libpgm,pgmpy,networkx,graphviz,pybnl,pytest

In [ ]:
%matplotlib inline
import numpy as np, pandas as pd, xarray as xr, matplotlib.pyplot as plt, seaborn as sns
import sklearn, sklearn.pipeline, sklearn.model_selection, sklearn.naive_bayes
import networkx as nx, graphviz, networkx.algorithms.dag
import random
import itertools, collections

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(edgeitems=10)
np.set_printoptions(suppress=True)
np.core.arrayprint._line_width = 180

sns.set()

In [ ]:
from IPython.display import display, HTML

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        if type(df) == np.ndarray:
            df = pd.DataFrame(df)
        html_str+=df.to_html()
    html_str = html_str.replace('table','table style="display:inline"')
    # print(html_str)
    display_html(html_str,raw=True)

CSS = """
.output {
    flex-direction: row;
}
"""

def display_graphs_side_by_side(*args):
    html_str='<table><tr>'
    for g in args:
        html_str += '<td>'
        html_str += g._repr_svg_()
        html_str += '</td>'
    html_str += '</tr></table>'
    display_html(html_str,raw=True)
    

display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport pybnl.bn

In [ ]:
%aimport dsbasics.bin

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'C')

import rpy2, rpy2.rinterface, rpy2.robjects, rpy2.robjects.packages, rpy2.robjects.lib, rpy2.robjects.lib.grid, \
    rpy2.robjects.lib.ggplot2, rpy2.robjects.pandas2ri, rpy2.interactive.process_revents, \
    rpy2.interactive, rpy2.robjects.lib.grdevices
# rpy2.interactive.process_revents.start()
rpy2.robjects.pandas2ri.activate()

In [ ]:
rpackageversionfn = rpy2.robjects.r('packageVersion')
print(rpackageversionfn("bnlearn")[0])
print(rpackageversionfn("gRain")[0])

# House Prices in Ames, Iowa

* [Ames, Iowa: Alternative to the Boston Housing Data as an End of Semester Regression Project](http://ww2.amstat.org/publications/jse/v19n3/decock.pdf)
  * [AmesResidential.pdf](https://ww2.amstat.org/publications/jse/v19n3/decock/AmesResidential.pdf)
  * [DataDocumentation.txt](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt)
  * [AmesHousing.txt](https://ww2.amstat.org/publications/jse/v19n3/decock/AmesHousing.txt)
  * [AmesHousing.xls](http://www.amstat.org/publications/jse/v19n3/decock/AmesHousing.xls)
  * Also on [kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)

The below example reproduces the example from chapter 5 (page 79) in [Bayesian Networks and BayesiaLab: A Practical Introduction for Researchers](https://www.amazon.com/Bayesian-Networks-BayesiaLab-Introduction-Researchers/dp/0996533303).

In [ ]:
df = pd.read_csv('./AmesHousing.txt.gz', sep='\t', index_col=0)
df['MS SubClass'] = df['MS SubClass'].apply(lambda x: '{0:0>3}'.format(x))
df.iloc[:5,:15]

In [ ]:
df.columns

In [ ]:
discrete_non_null, discrete_with_null, continuous_non_null, continuous_with_null, levels_map = pybnl.bn.discrete_and_continuous_variables_with_and_without_nulls(df, cutoff=30)
# discrete_non_null, discrete_with_null, continuous_non_null, continuous_with_null, levels_map

In [ ]:
ddf = df.copy()
#cat_columns = ['Alley', 'Bedroom AbvGr', 'Bldg Type', 'Bsmt Cond', ]
cat_columns = [
    'MS SubClass', 'MS Zoning', 'Street', 'Alley', 'Land Contour', 'Lot Config', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type', 'House Style',
    'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Foundation', 'Heating', 'Central Air', 'Garage Type', 'Misc Feature', 'Sale Type', 'Sale Condition'
] + [
    'Overall Qual', 'Overall Cond'
]
cat_columns_ordinal = [
    ('Lot Shape',      ['Reg','IR1','IR2','IR3']),
    ('Utilities',      ['AllPub','NoSewr','NoSeWa','ELO']),
    ('Land Slope',     ['Gtl', 'Mod', 'Sev']),
    ('Exter Qual',     ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
    ('Exter Cond',     ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
    ('Bsmt Qual',      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
    ('Bsmt Cond',      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
    ('Bsmt Exposure',  ['Gd', 'Av', 'Mn', 'No', 'NA']),
    ('BsmtFin Type 1', ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA']),
    ('BsmtFin Type 2', ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA']),
    ('Heating QC',     ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
    ('Electrical',     ['SBrkr', 'FuseA', 'FuseF', 'FuseP', 'Mix']),
    ('Kitchen Qual',   ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
    ('Functional',     ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal']),
    ('Fireplace Qu',   ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
    ('Garage Finish',  ['Fin', 'RFn', 'Unf', 'NA']),
    ('Garage Qual',    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
    ('Garage Cond',    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
    ('Paved Drive',    ['Y', 'P', 'N']),
    ('Pool QC',        ['Ex', 'Gd', 'TA', 'Fa', 'NA']),
    ('Fence',          ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA']),
]

continuous_columns = [
    'Lot Frontage', 'Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch',
    'Screen Porch', 'Pool Area', 'Misc Val', 'SalePrice'
]
discrete_columns = ['Year Built', 'Year Remod/Add', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'TotRms AbvGrd', 'Fireplaces', 'Garage Yr Blt', 'Garage Cars', 'Mo Sold', 'Yr Sold', 'Bedroom AbvGr', 'Kitchen AbvGr']# do not exist: 'Bedroom',  'Kitchen'

for col in cat_columns:
    levels = levels_map[col]
    # print('col: {}'.format(col))
    # if all([isinstance(level, (int, float)) for level in levels]):
    if all([np.issubdtype(type(level), np.number) for level in levels]):
        # print('int, float column: {}'.format(col))
        levels = sorted(levels)
        ddf[col] = df[col].astype(pd.api.types.CategoricalDtype(levels, ordered=True))
    else:
        ddf[col] = df[col].astype(pd.api.types.CategoricalDtype(levels, ordered=False))

for col, levels in cat_columns_ordinal:
    ddf[col] = df[col].astype(pd.api.types.CategoricalDtype(levels[::-1], ordered=True))

for col in continuous_columns:
    ddf[col] = df[col].astype(float)

for col in discrete_columns:
    if pd.isnull(df[col]).any():
        ddf[col] = df[col].astype(float)
    else:
        ddf[col] = df[col].astype(int)
    
# col   = 'Alley'
# ddf[col]
# ddf[~pd.isnull(ddf[col])][col]
# value = np.nan
# ddf.loc[df[col]==value,col]

[Working with Pandas: Fixing messy column names](https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd)

In [ ]:
ddf.columns = ddf.columns.str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [ ]:
type(ddf.columns)

In [ ]:
ddf.head()

In [ ]:
# ddf.to_hdf('AmesHousing.h5', 'AmesHousing',format='table', append=False)

In [ ]:
# pd.read_hdf('AmesHousing.h5', 'AmesHousing').head()

## Treating Filtered Values ('FV')

See page 84 in "Bayesian Networks and BayesiaLab"

### Bsmt fields

In [ ]:
bsmt_fields_ = ['Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Bsmt Full Bath', 'Bsmt Half Bath', 
               'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF']
bsmt_fields = pd.Index(bsmt_fields_).str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
ddf[bsmt_fields].query('Bsmt_Qual == "NA"')
# ddf[ddf['Bsmt_Qual'] == 'NA'][bsmt_fields]
# df[bsmt_fields_][df['Bsmt Qual'] == 'NA']

It seems that there are no filtered values for 'Bsmt' fields, e.g. each home contains a basement.

[Querying for NaN and other names in Pandas](https://stackoverflow.com/questions/26535563/querying-for-nan-and-other-names-in-pandas)

In [ ]:
ddf[bsmt_fields][pd.isnull(ddf.Bsmt_Qual)].head()

In [ ]:
df[bsmt_fields_][pd.isnull(df['Bsmt Qual'])].head()

But there are quite a lot of 'NaN' entries. Not sure why the data description contains an "NA" value as "No Basement", but no actual data-set uses it. Most likely these values are supposed to be "NA"

In [ ]:
bsmt_na_fields = ['Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure', 'BsmtFin_Type_1', 'BsmtFin_Type_2']
ddf.loc[pd.isnull(ddf.Bsmt_Qual), bsmt_na_fields] = "NA"
ddf[bsmt_fields].query('Bsmt_Qual == "NA"').head()

### Electrical field

In [ ]:
ddf.Electrical.value_counts(dropna=False)

In [ ]:
ddf[pd.isnull(ddf.Electrical)]

The one NaN value seems to be a missing value

### Fireplaces

In [ ]:
fireplaces_fields_ = ['Fireplaces', 'Fireplace Qu']
fireplaces_fields = pd.Index(fireplaces_fields_).str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
ddf[fireplaces_fields].query('Fireplaces == 0').head()

In [ ]:
ddf.loc[ddf.Fireplaces == 0,['Fireplace_Qu']] = 'NA'
ddf[fireplaces_fields].query('Fireplaces == 0').head()

### Garage fields

In [ ]:
garage_fields_ = ['Garage Type', 'Garage Finish', 'Garage Cars', 'Garage Qual', 'Garage Cond', 'Garage Yr Blt', 'Garage Area']
garage_fields = pd.Index(garage_fields_).str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
ddf[garage_fields][pd.isnull(ddf.Garage_Type)].head()

In [ ]:
ddf['Garage_Type'] = ddf['Garage_Type'].astype(str)\
    .astype(pd.api.types.CategoricalDtype(set(list(ddf.Garage_Type.dtype.categories) + ['NA'])))
ddf.Garage_Type.dtype.categories

In [ ]:
ddf.loc[pd.isnull(ddf.Garage_Type),['Garage_Type', 'Garage_Finish', 'Garage_Qual', 'Garage_Cond']] = 'NA'
ddf.loc[ddf.Garage_Type == 'NA',['Garage_Yr_Blt']] = -1.0
#ddf[garage_fields][pd.isnull(ddf.Garage_Yr_Blt)]
#ddf['Garage_Yr_Blt'] = ddf['Garage_Yr_Blt'].astype(int)
ddf[garage_fields][ddf.Garage_Type == 'NA'].head()

### Mas Vnr fields

In [ ]:
mas_vnr_fields_ = ['Mas Vnr Type', 'Mas Vnr Area']
mas_vnr_fields = pd.Index(mas_vnr_fields_).str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
ddf[mas_vnr_fields][pd.isnull(ddf.Mas_Vnr_Type)].head()

In [ ]:
ddf.Mas_Vnr_Type.dtype

In [ ]:
ddf.loc[pd.isnull(ddf.Mas_Vnr_Type), ['Mas_Vnr_Type']] = 'None'
ddf.loc[ddf.Mas_Vnr_Type == 'None', ['Mas_Vnr_Area']] = 0.0

In [ ]:
ddf[mas_vnr_fields][ddf.Mas_Vnr_Type == 'None'].head()

### Pool fields

In [ ]:
pool_fields_ = ['Pool QC', 'Pool Area']
pool_fields = pd.Index(pool_fields_).str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
ddf[pool_fields][pd.isnull(ddf.Pool_QC)].head()

In [ ]:
ddf.loc[pd.isnull(ddf.Pool_QC), ['Pool_QC']] = 'NA'
ddf[pool_fields][ddf.Pool_QC == 'NA'].head()

### Fence field

In [ ]:
fence_fields_ = ['Fence']
fence_fields = pd.Index(fence_fields_).str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
ddf.loc[pd.isnull(ddf.Fence), ['Fence']] = 'NA'
ddf[fence_fields][ddf.Fence == 'NA'].head()

### Misc Feature field

In [ ]:
misc_feature_fields_ = ['Misc Feature']
misc_feature_fields = pd.Index(misc_feature_fields_).str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
ddf['Misc_Feature'] = ddf['Misc_Feature'].astype(str)\
    .astype(pd.api.types.CategoricalDtype(set(list(ddf.Misc_Feature.dtype.categories) + ['NA'])))
ddf.loc[pd.isnull(ddf.Misc_Feature), ['Misc_Feature']] = 'NA'
ddf[misc_feature_fields][ddf.Misc_Feature == 'NA'].head()

### Check remaining nan fields

In [ ]:
_, discrete_with_null_, _, continuous_with_null_, _ = pybnl.bn.discrete_and_continuous_variables_with_and_without_nulls(ddf, cutoff=30)
discrete_with_null_, continuous_with_null_

In [ ]:
ddf[bsmt_fields][pd.isnull(ddf.Bsmt_Exposure)]

Just set the few NaN values to 'No'

In [ ]:
ddf.loc[pd.isnull(ddf.Bsmt_Exposure),['Bsmt_Exposure']] = 'No'

In [ ]:
ddf[bsmt_fields][pd.isnull(ddf.BsmtFin_Type_2)]

Just set the few NaN values to 'Unf'

In [ ]:
ddf.loc[pd.isnull(ddf.BsmtFin_Type_2),['BsmtFin_Type_2']] = 'Unf'

In [ ]:
ddf[bsmt_fields][pd.isnull(ddf.Bsmt_Full_Bath)]

In [ ]:
ddf[bsmt_fields][pd.isnull(ddf.Bsmt_Half_Bath)]

Just set all the NaN values to 0.0

In [ ]:
ddf.loc[pd.isnull(ddf.Bsmt_Full_Bath),['Bsmt_Full_Bath','Bsmt_Half_Bath','BsmtFin_SF_1','BsmtFin_SF_2','Bsmt_Unf_SF','Total_Bsmt_SF']] = [0.0,0.0,0.0,0.0,0.0,0.0]

In [ ]:
ddf[['Electrical']][pd.isnull(ddf.Electrical)]

Just set this single NaN value to 'Mix'

In [ ]:
ddf.loc[pd.isnull(ddf.Electrical),['Electrical']] = 'Mix'

The remaining NaN garage fields seem to be really missing values so don't touch them.

In [ ]:
ddf[garage_fields][pd.isnull(ddf.Garage_Finish)]

In [ ]:
ddf[garage_fields][pd.isnull(ddf.Garage_Cars)]

In [ ]:
ddf[garage_fields][pd.isnull(ddf.Garage_Qual)]

In [ ]:
ddf[garage_fields][pd.isnull(ddf.Garage_Cond)]

The Lot_Frontage NaN fields seem to be really missing values so don't touch them.

In [ ]:
ddf[pd.isnull(ddf.Lot_Frontage)].head()

In [ ]:
_, discrete_with_null_, _, continuous_with_null_, _ = pybnl.bn.discrete_and_continuous_variables_with_and_without_nulls(ddf, cutoff=30)
discrete_with_null_, continuous_with_null_

## Binning / Discretization

In [ ]:
ddf1 = ddf.copy()

### The target variable is SalePrice

In [ ]:
ddf.SalePrice.describe()

In [ ]:
ddf1.SalePrice = pd.cut(ddf.SalePrice, [0.0,75000.0, 150000.0, 225000.0, 300000.0,np.PINF], right=False)
ddf1['SalePrice'].value_counts()

### Continuous, discrete and ordinal variables

In [ ]:
continuous_columns_without_sale_price = list(set(continuous_columns) - set(['SalePrice']))

In [ ]:
target_variable_decision_tree_binning_variables_ = [c for c,r in cat_columns_ordinal] + continuous_columns_without_sale_price + discrete_columns + ['Overall Qual', 'Overall Cond']
target_variable_decision_tree_binning_variables = pd.Index(target_variable_decision_tree_binning_variables_).str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
target_variable_decision_tree_binning_variables

In [ ]:
tvbt = dsbasics.bin.TargetVariableDecisionTreeBinTransformer0(max_leaf_nodes=3)
ddf1.loc[:,target_variable_decision_tree_binning_variables] = \
    tvbt.fit_transform(ddf[target_variable_decision_tree_binning_variables], ddf1.SalePrice)
ddf1.head()

In [ ]:
len(df.columns),len(ddf1.columns)

In [ ]:
ddf['Overall_Qual'].value_counts().sort_index().plot(kind='bar');

In [ ]:
ddf1['Overall_Qual'].value_counts().sort_index().plot(kind='bar');

### Convert interval indices to strings

In [ ]:
ddf1 = pybnl.bn.convert_interval_index_categories_to_string_categories(ddf1,inplace=False)

In [ ]:
ddf1.Bsmt_Full_Bath.dtype

### Rename columns to fit with R conventions

In [ ]:
ddf1 = ddf1.rename(columns={
    "1st_Flr_SF":"X1st_Flr_SF",
    "2nd_Flr_SF":"X2nd_Flr_SF",
    "3Ssn_Porch":"X3Ssn_Porch",
    "Year_Remod/Add":"Year_Remod_Add"
})

### Drop unused columns

In [ ]:
if any(ddf1.columns.isin(['PID'])):
    ddf1.drop('PID', axis=1, inplace=True)
# ddf1.columns

#### Drop the 'Alley' column, which contains most null values and will not contribute to the quality of the fit

In [ ]:
if any(ddf1.columns.isin(['Alley'])):
    ddf1.drop('Alley', axis=1, inplace=True)
# ddf1.columns

The remaining columns with null values are:

In [ ]:
_, discrete_with_null_, _, continuous_with_null_, _ = pybnl.bn.discrete_and_continuous_variables_with_and_without_nulls(ddf1, cutoff=30)
discrete_with_null_, continuous_with_null_

The statistics of the NaN values are as follows. Only the column `Lot_Frontage` still has relevant null values, e.g. we could simply filter out the 2 `Garage` rows that contain null values:

In [ ]:
display_side_by_side(
    pd.DataFrame(ddf1.Lot_Frontage.value_counts(dropna=False)),
    pd.DataFrame(ddf1.Garage_Yr_Blt.value_counts(dropna=False)),
    pd.DataFrame(ddf1.Garage_Finish.value_counts(dropna=False)),
    pd.DataFrame(ddf1.Garage_Cars.value_counts(dropna=False)),
    pd.DataFrame(ddf1.Garage_Area.value_counts(dropna=False)),
    pd.DataFrame(ddf1.Garage_Qual.value_counts(dropna=False)),
    pd.DataFrame(ddf1.Garage_Cond.value_counts(dropna=False))
)

In [ ]:
null_value_idx = pd.isnull(ddf1.Lot_Frontage) | pd.isnull(ddf1.Garage_Yr_Blt) | pd.isnull(ddf1.Garage_Finish) | \
    pd.isnull(ddf1.Garage_Cars) | pd.isnull(ddf1.Garage_Area) | pd.isnull(ddf1.Garage_Qual) | pd.isnull(ddf1.Garage_Cond)
null_value_idx.value_counts()

In [ ]:
ddf_without_null_values = ddf1[~null_value_idx]

In [ ]:
ddf_with_null_values = ddf1

## Naive Bayes Classifier

### bnlearn by hand

#### Define net

In [ ]:
dg = nx.DiGraph()

dg.add_nodes_from(ddf1.columns.values)

in_vars = ddf1.columns.values[:-1]
out_var = ddf1.columns.values[-1:]
dg.add_edges_from(list(itertools.product(out_var, in_vars)))

In [ ]:
ns = pybnl.bn.digraph2netstruct(dg)
# ns.dot()
display(HTML(ns.dot()._repr_svg_()))

In [ ]:
nbn1 = pybnl.bn.NetAndDataDiscreteBayesNetwork(ldf=ddf_without_null_values, dg=dg, predict_var='SalePrice')
nbn1.fit();

In [ ]:
N=20 # restricted to speed up runtime of notebook
y_pred = nbn1.predict(ddf_without_null_values.iloc[:N,:-1])

In [ ]:
y_true = ddf_without_null_values.iloc[:N,-1:]

In [ ]:
sklearn.metrics.accuracy_score(y_true, y_pred)

In [ ]:
tmp_cm = sklearn.metrics.confusion_matrix(y_true, y_pred)
tmp_cm

In [ ]:
np.diag(tmp_cm).sum()/tmp_cm.sum()

In [ ]:
nbn1.arc_strength_info().head()

In [ ]:
nbn1.arc_strength_info().tail()

In [ ]:
display(HTML(nbn1.dot()._repr_svg_()))

### bnlearn via naive.bayes

In [ ]:
mbn = pybnl.bn.MultinomialNB()
mbn.fit(ddf_without_null_values[in_vars], ddf_without_null_values[out_var[0]])

In [ ]:
nbn1.arc_strength_info().head()

In [ ]:
tmp_df = pd.DataFrame([
    mbn.predict(ddf_without_null_values[in_vars]),
    ddf_without_null_values[out_var[0]].reset_index(drop=True),    
])
tmp_df.head()

In [ ]:
sklearn.metrics.accuracy_score(tmp_df.iloc[:,1], tmp_df.iloc[:,0])

In [ ]:
tmp_cm = sklearn.metrics.confusion_matrix(tmp_df.iloc[:,1], tmp_df.iloc[:,0])
tmp_cm

In [ ]:
# display(HTML(mbn.dot()._repr_svg_()))

In [ ]:
mbn.predict_proba(ddf_without_null_values[in_vars]).head()

In [ ]:
tmp_df = pd.DataFrame([
    mbn.predict(ddf_without_null_values[in_vars]),
    mbn.predict_proba(ddf_without_null_values[in_vars]).max(axis=1).reset_index(drop=True),
    ddf_without_null_values[out_var[0]].reset_index(drop=True),
    mbn.predict_proba(
        ddf_without_null_values[in_vars]
    ).values[
        np.array(range(len(ddf_without_null_values))).reshape(-1,1),
        ddf_without_null_values[out_var[0]].values.codes.reshape(-1,1)
    ].reshape(-1)
])
tmp_df.head()

### sklearn MultinomialNB

In [ ]:
clf = sklearn.naive_bayes.MultinomialNB()
X_ = ddf_without_null_values[in_vars]
X  = X_.apply(lambda x: x.cat.codes, axis=0)
y_ = ddf_without_null_values[out_var[0]]
y  = y_.cat.codes
clf.fit(X, y)
# 

In [ ]:
tmp_df = pd.DataFrame([
    pybnl.bn.from_codes_to_category(
        clf.predict(X),y_.dtype),
    clf.predict_proba(X).max(axis=1),
    y_.reset_index(drop=True),    
    clf.predict_proba(X)[np.array(range(len(y))).reshape(-1,1),y.values.reshape(-1,1)].reshape(-1)
])
tmp_df.head()

In [ ]:
y_.dtype.categories

In [ ]:
sklearn.metrics.accuracy_score(y, clf.predict(X))

In [ ]:
sklearn.metrics.precision_score(y, clf.predict(X), average='micro')

In [ ]:
sklearn.metrics.recall_score(y, clf.predict(X), average='micro')

In [ ]:
sklearn.metrics.f1_score(y, clf.predict(X), average='micro')

In [ ]:
# sklearn.metrics.roc_auc_score(y, tmp_df.iloc[:,3], average='micro')

In [ ]:
# vertically: true label
# horizontally: predicted label
tmp_cm = sklearn.metrics.confusion_matrix(y, clf.predict(X))
tmp_cm

In [ ]:
np.diag(tmp_cm).sum()/tmp_cm.sum()

* [Tutorial on MultinomialNB](http://universityofbigdata.net/competition/tutorial/5681717746597888?lang=en)
* [Vectorization, Multinomial Naive Bayes Classifier and Evaluation](https://www.ritchieng.com/machine-learning-multinomial-naive-bayes-vectorization/)
* [Computing cross-validated metrics](http://scikit-learn.org/stable/modules/cross_validation.html)

In [ ]:
clf_cv = sklearn.naive_bayes.MultinomialNB(alpha=1.0)
# scores = sklearn.model_selection.cross_val_score(clf_cv, X, y, cv=10, scoring='accuracy')
# cv = sklearn.model_selection.KFold(n_splits=10, shuffle=False, random_state=0)
cv = sklearn.model_selection.StratifiedKFold(n_splits=10, shuffle=False, random_state=0)
# cv = sklearn.model_selection.ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
scores = sklearn.model_selection.cross_val_score(clf_cv, X, y, cv=cv, scoring='accuracy')
np.mean(scores), np.std(scores)

In [ ]:
y_.dtype.categories

In [ ]:
bbt1 = dsbasics.bin.BayesianBlocksBinTransformer(p0=0.01)
bbt1.fit(ddf['SalePrice'].values.reshape(-1,1))
bbt2 = dsbasics.bin.BayesianBlocksBinTransformer(neg_ln_gamma=30)
bbt2.fit(ddf['SalePrice'].values.reshape(-1,1))
fig=plt.figure(figsize=(8, 4), dpi= 80, facecolor='w', edgecolor='k')
ax = plt.subplot()
ax.hist(ddf['SalePrice'], bins=200, histtype='stepfilled', alpha=0.4, density=True);
ax.hist(ddf['SalePrice'], bins=bbt2.bins_[0], histtype='step', color='black', density=True);

In [ ]:
ddf_without_null_values_saleprice = ddf.loc[ddf_without_null_values.index]['SalePrice']

tmp_saleprice_category_to_mean_mapping = \
    ddf_without_null_values_saleprice.groupby(ddf_without_null_values['SalePrice']).mean()
tmp_saleprice_category_to_mean_mapping_dict = \
    dict(zip(tmp_saleprice_category_to_mean_mapping.index, tmp_saleprice_category_to_mean_mapping.values))
tmp_saleprice_category_to_mean_mapping

In [ ]:
tmp_y_pred = ddf_without_null_values['SalePrice'].map(tmp_saleprice_category_to_mean_mapping)
tmp_y_real = ddf_without_null_values_saleprice
tmp_pred_vs_real_df = pd.concat([tmp_y_pred, tmp_y_real], axis=1)
tmp_pred_vs_real_df.columns = ['pred', 'real']
tmp_pred_vs_real_df.head()

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(tmp_pred_vs_real_df.real, tmp_pred_vs_real_df.pred))

In [ ]:
sklearn.metrics.median_absolute_error(tmp_pred_vs_real_df.real, tmp_pred_vs_real_df.pred)

In [ ]:
sklearn.metrics.mean_absolute_error(tmp_pred_vs_real_df.real, tmp_pred_vs_real_df.pred)

In [ ]:
sklearn.metrics.explained_variance_score(tmp_pred_vs_real_df.real, tmp_pred_vs_real_df.pred)

## End to end sklearn pipeline

In [ ]:
df.iloc[:5,:15]

In [ ]:
cat_columns = [
    'MS SubClass', 'MS Zoning', 'Street', 'Land Contour', 'Lot Config', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', #'Alley', 
    'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Foundation', 'Heating', 'Central Air', 'Garage Type', 'Misc Feature', 'Sale Type', 'Sale Condition'
] + [
    'Overall Qual', 'Overall Cond'
]
cat_columns_ordinal = [
    ('Lot Shape',      ['Reg','IR1','IR2','IR3']),
    ('Utilities',      ['AllPub','NoSewr','NoSeWa','ELO']),
    ('Land Slope',     ['Gtl', 'Mod', 'Sev']),
    ('Exter Qual',     ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
    ('Exter Cond',     ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
    ('Bsmt Qual',      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
    ('Bsmt Cond',      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
    ('Bsmt Exposure',  ['Gd', 'Av', 'Mn', 'No', 'NA']),
    ('BsmtFin Type 1', ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA']),
    ('BsmtFin Type 2', ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA']),
    ('Heating QC',     ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
    ('Electrical',     ['SBrkr', 'FuseA', 'FuseF', 'FuseP', 'Mix']),
    ('Kitchen Qual',   ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
    ('Functional',     ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal']),
    ('Fireplace Qu',   ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
    ('Garage Finish',  ['Fin', 'RFn', 'Unf', 'NA']),
    ('Garage Qual',    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
    ('Garage Cond',    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
    ('Paved Drive',    ['Y', 'P', 'N']),
    ('Pool QC',        ['Ex', 'Gd', 'TA', 'Fa', 'NA']),
    ('Fence',          ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA']),
]

continuous_columns = [
    'Lot Frontage', 'Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch',
    'Screen Porch', 'Pool Area', 'Misc Val', 'SalePrice'
]
discrete_columns = ['Year Built', 'Year Remod/Add', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'TotRms AbvGrd', 'Fireplaces', 'Garage Yr Blt', 'Garage Cars', 'Mo Sold', 'Yr Sold', 'Bedroom AbvGr', 'Kitchen AbvGr']# do not exist: 'Bedroom',  'Kitchen'


In [ ]:
cat_columns_NA = [
    ('Misc Feature', ['Shed', 'Gar2', 'Othr', 'TenC', 'Elev', 'NA']),
    ('Garage Type', ['Attchd', 'Detchd', 'BuiltIn', 'Basment', '2Types', 'CarPort', 'NA']),
]

In [ ]:
df['Garage Type'].value_counts(dropna=False)

In [ ]:
tmp_cat_columns_ordinal = [e for e,_ in cat_columns_ordinal]
# tmp_cat_columns_ordinal

In [ ]:
tmp_cat_columns_NA = [(e, 'NA') for e,_ in cat_columns_NA]
tmp_cat_columns_NA += [('Bsmt Qual', 'NA'), ('Bsmt Cond', 'NA'), ('Bsmt Exposure', 'NA'), ('BsmtFin Type 1', 'NA'), ('BsmtFin Type 2', 'NA')] # 
tmp_cat_columns_NA += [('Fireplace Qu', 'NA')]
tmp_cat_columns_NA += [('Garage Finish', 'NA'), ('Garage Qual', 'NA'), ('Garage Cond', 'NA'), ('Garage Yr Blt', -1.0)]
tmp_cat_columns_NA += [('Mas Vnr Type', 'None'), ['Mas Vnr Area', 0.0]]
tmp_cat_columns_NA += [('Pool QC', 'NA')]
tmp_cat_columns_NA += [('Fence', 'NA')]
tmp_cat_columns_NA += [('Misc Feature', 'NA')]
tmp_cat_columns_NA += [('Bsmt Exposure', 'No'), ('BsmtFin Type 2', 'Unf'), ('Bsmt Full Bath', 0.0), ('Bsmt Half Bath', 0.0), ('BsmtFin SF 1', 0.0), ('BsmtFin SF 2', 0.0), ('Bsmt Unf SF', 0.0), ('Total Bsmt SF', 0.0)]
tmp_cat_columns_NA += [('Electrical', 'Mix')]
tmp_cat_columns_NA

In [ ]:
tmp_cat_columns_ordinal_1 = [(e, l[::-1]) for e,l in cat_columns_ordinal + cat_columns_NA]
tmp_levels_map = dict(tmp_cat_columns_ordinal_1)
# tmp_levels_map

In [ ]:
metadata = {}
it = dsbasics.bin.MetaDataInitTransformer(metadata)

dc = dsbasics.bin.DropColumnTransformer(['PID', 'Alley'])

ct = dsbasics.bin.CategoricalTransformer(
    categorical_columns = cat_columns,
    ordered_categorical_columns = tmp_cat_columns_ordinal, 
    discrete_columns = discrete_columns, 
    continuous_columns = continuous_columns, 
    levels_map = tmp_levels_map,
    metadata=metadata
)

tvbt = dsbasics.bin.TargetVariableDecisionTreeBinTransformer(
    max_leaf_nodes=3, 
    binning_variables=target_variable_decision_tree_binning_variables_,
    metadata=metadata
)

pc = dsbasics.bin.PandasCutBinTransformer({'SalePrice': [75000.0, 150000.0, 225000.0, 300000.0]}, metadata=metadata)

nt = dsbasics.bin.NullToNATransformer(null_to_NA_columns = tmp_cat_columns_NA, metadata=metadata)

lt = dsbasics.bin.CategoryLevelsAsStringsTransformer(metadata=metadata)

fn = dsbasics.bin.FilterNullTransformer(metadata=metadata)

mbn = pybnl.bn.MultinomialNB()

pl = sklearn.pipeline.Pipeline(
    memory=None,
    steps=[
        ('init', it),
        ('drop_columns', dc),
        ('ct', ct),
        ('null_to_NA', nt),
        ('target_variable_binning', pc),
        ('decision_tree_discretization', tvbt),
        ('levels_as_strings', lt),
        ('filter_null_rows', fn),
])

tmp = pl.fit_transform(df.iloc[:,:-1], df.SalePrice)
tmp.head()

In [ ]:
ddf_without_null_values.shape

In [ ]:
tmp.shape

In [ ]:
for i in range(2438):
    idx = ddf_without_null_values.index[i]
    row1 = ddf_without_null_values.iloc[:,:-1].loc[idx].reset_index(drop=True)
    row2 = tmp.loc[idx].reset_index(drop=True)
    if not row1.equals(row2):
        print(idx)
        break

eql=(row1==row2)
tmp_cmp = pd.DataFrame(collections.OrderedDict(nm=tmp.columns,row1=row1, row2=row2, eql=eql))
tmp_cmp[~tmp_cmp.eql]

In [ ]:
tmp_df = pl.named_steps['target_variable_binning'].df
tmp_cmp = pd.DataFrame(collections.OrderedDict(df=df['Mas Vnr Area'],tmp=tmp_df.Mas_Vnr_Area))
tmp_cmp['eql'] = tmp_cmp.df == tmp_cmp.tmp
tmp_cmp[~tmp_cmp.eql].head()

In [ ]:
tmp_df[['Mas_Vnr_Area', 'SalePrice']].head()

In [ ]:
tvbt_t1 = dsbasics.bin.TargetVariableDecisionTreeBinTransformer(
    max_leaf_nodes=3, 
    binning_variables=['Mas_Vnr_Area'],
)
tvbt_t1.fit_transform(tmp_df, tmp_df.iloc[:,-1]).Mas_Vnr_Area.head()

In [ ]:
tmp.Mas_Vnr_Area.head()

In [ ]:
ddf_without_null_values.Mas_Vnr_Area.head()

In [ ]:
df['Mas Vnr Area'].value_counts(dropna=False).reset_index().sort_values(['index']).head()

In [ ]:
ddf['Mas_Vnr_Area'].value_counts(dropna=False).reset_index().sort_values(['index']).head()

In [ ]:
tmp_df['Mas_Vnr_Area'].value_counts(dropna=False).reset_index().sort_values(['index']).head()

In [ ]:
# tvbt_t2 = dsbasics.bin.TargetVariableDecisionTreeBinTransformer0(max_leaf_nodes=3)
# ddf1.loc[:,target_variable_decision_tree_binning_variables] = \
#     tvbt_t2.fit_transform(ddf[target_variable_decision_tree_binning_variables], ddf1.SalePrice)
# ddf1.head()

In [ ]:
pl.named_steps['levels_as_strings'].df.Bsmt_Qual.dtype

In [ ]:
null_value_idx = pd.isnull(ddf1.Lot_Frontage) | pd.isnull(ddf1.Garage_Yr_Blt) | pd.isnull(ddf1.Garage_Finish) | \
    pd.isnull(ddf1.Garage_Cars) | pd.isnull(ddf1.Garage_Area) | pd.isnull(ddf1.Garage_Qual) | pd.isnull(ddf1.Garage_Cond)
null_value_idx.value_counts()

In [ ]:
pl.named_steps['filter_null_rows'].null_fields

In [ ]:
pl.named_steps['levels_as_strings'].df.SalePrice.dtype

In [ ]:
tmp.Garage_Type.dtype

In [ ]:
tmp.Garage_Type.value_counts(dropna=False)

In [ ]:
tmp.Overall_Qual.dtype

In [ ]:
tmp.Lot_Area.dtype